In [1]:
from langchain_groq import ChatGroq

In [5]:
import os
from langchain_groq import ChatGroq  # or your specific library if different

# Make sure to set this in your environment variables before running
# Example: export GROQ_API_KEY="your_api_key_here"
groq_api_key = "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"

llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="llama3-70b-8192"  # updated model naming (based on actual Groq documentation if needed)
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)


That's an easy one!

The first person to set foot on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the Moon.


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/110690555461018310-software-engineer-iii-infrastructure-core")
page_data = loader.load().pop().page_content
print(page_data)

Software Engineer III, Infrastructure, Core — Google CareersCareersSkip navigation linkshomehomeHomeHomework_outlinework_outlineJobsJobsnoogler_hatnoogler_hatStudentsStudentsgooglegoogleHow we workHow we workhandymanhandymanHow we hireHow we hireperson_outlineperson_outlineYour careerYour careerhelp_outlineHelp linkfeedbackSend feedbackmore_vert HelpSend FeedbackSign inCareershomeHomework_outlineJobsexpand_morenoogler_hatStudentsexpand_moregoogleHow we workexpand_morehandymanHow we hireexpand_moreperson_outlineYour careerexpand_morejob detailsarrow_backBack to jobs searchJobs search results3,157  jobs matchedSoftware Engineer III, Infrastructure, CoreBengaluru, Karnataka, IndiaPersistent Disk Capacity Team LeaderKirkland, WA, USASenior Software Engineer, Android PartnerWarsaw, PolandSenior Software Engineer, Core Data, Data IndexingBengaluru, Karnataka, IndiaSoftware Engineer III, Google CloudBengaluru, Karnataka, India; Pune, Maharashtra, IndiaProduct Operations Lead, Google Play Apps

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer III, Infrastructure, Core',
  'experience': 'Mid',
  'skills': ['software development',
   'data structures',
   'algorithms',
   'infrastructure',
   'distributed systems',
   'networks',
   'compute technologies',
   'storage',
   'hardware architecture',
   'accessible technologies'],
  'description': 'Write product or system development code. Participate in, or lead design reviews with peers and stakeholders to decide amongst available technologies. Review code developed by other developers and provide feedback to ensure best practices. Contribute to existing documentation or educational content and adapt content based on product/program updates and user feedback. Triage product or system issues and debug/track/resolve by analyzing the sources of issues and the impact on hardware, network, or service operations and quality.'}]

In [10]:
type(json_res)

list

In [11]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [17]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [19]:
links = collection.query(query_texts=job[0]['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'ht

In [20]:
job = json_res
job[0]['skills']

['software development',
 'data structures',
 'algorithms',
 'infrastructure',
 'distributed systems',
 'networks',
 'compute technologies',
 'storage',
 'hardware architecture',
 'accessible technologies']

In [21]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Infrastructure Development Services for Your Organization

Dear [Hiring Manager's Name],

I came across the job description for a Software Engineer III, Infrastructure, Core, and I believe AtliQ can be a great fit to support your organization's infrastructure development needs. Our team of experts has extensive experience in software development, data structures, algorithms, infrastructure, distributed systems, networks, compute technologies, storage, hardware architecture, and accessible technologies.

At AtliQ, we have a proven track record of delivering scalable and efficient solutions that cater to the unique requirements of our clients. Our team can help you write product or system development code, participate in design reviews, review code, and provide feedback to ensure best practices. We can also contribute to existing documentation or educational content and adapt it based on product/program updates and user feedback.

Our expertise in infrastructure developme